In [1]:
from nltk.tokenize.punkt import PunktSentenceTokenizer
article =  spark.read.text('/FileStore/tables/OneArticle.txt')
articlerdd = sc.textFile('/FileStore/tables/OneArticle.txt')

In [2]:
sentences = articlerdd.flatMap(lambda line: line.split("."))


In [3]:
sentences_count = sentences.count()


In [4]:
from collections import Counter
 
def bag_of_words(article):
  return Counter(word.lower().strip('.,') for word in sentence.split(' '))

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

c = CountVectorizer()
bow_array = c.fit_transform(sentences.take(sentences_count))
bow_array.toarray()

Out[10]: array([[0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 ...,
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

c = CountVectorizer()
bow_matrix = c.fit_transform(sentences.take(sentences_count))
bow_matrix

Out[11]: <69x372 sparse matrix of type '<class 'numpy.int64'>'
	with 737 stored elements in Compressed Sparse Row format>

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

normalized_matrix = TfidfTransformer().fit_transform(bow_matrix)

In [8]:
similarity_graph = normalized_matrix * normalized_matrix.T
similarity_graph.toarray()

Out[13]: array([[1. , 0.18620678, 0. , ..., 0.09888109, 0.13971366,
 0. ],
 [0.18620678, 1. , 0. , ..., 0. , 0.05147593,
 0. ],
 [0. , 0. , 0. , ..., 0. , 0. ,
 0. ],
 ...,
 [0.09888109, 0. , 0. , ..., 1. , 0.06344784,
 0. ],
 [0.13971366, 0.05147593, 0. , ..., 0.06344784, 1. ,
 0. ],
 [0. , 0. , 0. , ..., 0. , 0. ,
 0. ]])

In [9]:
import networkx as nx

nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
scores = nx.pagerank(nx_graph)

In [10]:
ranked = sorted(((scores[i],s) for i,s in enumerate(sentences.take(sentences_count))),
reverse=True)
print(ranked[0][1] + ". "+ranked[1][1] + ". "+ranked[2][1])

“That’s one of the things we try to make sure they have to understand: They have to be themselves. They have a higher average in June, but they have hit only 24 homers, and the inconsistent offense has put a strain on the pitching staff. The team asked Verrett to start Wednesday and gave Matz an extra day of rest after he received anti-inflammatory medication for the large bone spur in his left elbow